In [1]:
import numpy as np
import math
import random
#from scipy.stats import unitary_group as ug

In [2]:
def trap_bound(q, a, l):
    return 2 * a * ((2/(q**2 * (q**2 + 1))) ** (a//2 + l)) * ((2*q/(q**2 + 1)) ** l) / (1 - a * ((2*q/(q**2 + 1)) ** l)) * q ** (2*a + 4*l)

def partial_tr_bound(q, a, l):
    return 4 * (4 ** a) * a * ((2*q/(q**2 + 1)) ** (2*l))

In [3]:
#compute coefficients for entire circuit with product state input
#n should be even, l >= 2, q >= 2
def coeffs_product_state(q, n, l, lstep):
    #useful defs
    d = q**2
    w = q/(q**2 + 1)
    cn = 2**n
    cm = 2**(n//2)
    configs = {format(i, '0'+str(n)+'b'): i for i in range(cn)}
    lastconfig = {}
    for i in range(cm):
        rep = format(i, '0'+str(n//2)+'b')
        config = ''
        for j in range(n//2):
            config += rep[j] + rep[j]
        lastconfig[config] = configs[config]
    c = np.zeros(((l-1)*n//2 + 1, cn))
    for cii in lastconfig:
        c[0][lastconfig[cii]] = (1/d/(d+1))**(n//2)
    offset = -1
    nextconfig = {}
    for t in range(1, l):
        #print(t)
        offset *= -1
        for g in range(n//2 - 1):
            j = (t-1)*n//2 + g + 1
            #even layer: g, g-1 input
            #odd layer: g, g+1 input
            for config in lastconfig:
                if config[2*g + 1] == config[2*g + 1 + offset]:
                    nextconfig[config] = offset
                    c[j][configs[config]] += c[j - 1][configs[config]]
                elif offset > 0:
                    config1 = config[:2*g + 1] + '00' + config[2*g + 3:]
                    config2 = config[:2*g + 1] + '11' + config[2*g + 3:]
                    nextconfig[config1] = offset
                    nextconfig[config2] = offset
                    c[j][configs[config1]] += c[j - 1][configs[config]] * w
                    c[j][configs[config2]] += c[j - 1][configs[config]] * w
                else:
                    config1 = config[:2*g] + '00' + config[2*g + 2:]
                    config2 = config[:2*g] + '11' + config[2*g + 2:]
                    nextconfig[config1] = offset
                    nextconfig[config2] = offset
                    c[j][configs[config1]] += c[j - 1][configs[config]] * w
                    c[j][configs[config2]] += c[j - 1][configs[config]] * w
            #print(g)
            #print(lastconfig)
            #print(nextconfig)
            #print()
            lastconfig = nextconfig
            nextconfig = {}
        #last gate separately
        j = t*n//2
        g = -1
        for config in lastconfig:
            if config[2*g + 1] == config[2*g + 1 + offset]:
                nextconfig[config] = offset
                c[j][configs[config]] += c[j - 1][configs[config]]
            elif offset < 0:
                config1 = config[:-2] + '00'
                config2 = config[:-2] + '11'
                nextconfig[config1] = offset
                nextconfig[config2] = offset
                c[j][configs[config1]] += c[j - 1][configs[config]] * w
                c[j][configs[config2]] += c[j - 1][configs[config]] * w
            else:
                config1 = '0' + config[1:-1] + '0'
                config2 = '1' + config[1:-1] + '1'
                nextconfig[config1] = offset
                nextconfig[config2] = offset
                c[j][configs[config1]] += c[j - 1][configs[config]] * w
                c[j][configs[config2]] += c[j - 1][configs[config]] * w
        #print(lastconfig)
        #print(nextconfig)
        lastconfig = nextconfig
        nextconfig = {}
    #print(lastconfig)
    #print(c[-1])
    #print(np.sum(c[-1]) - c[-1][0] - c[-1][-1])
    return [c[lstep*i*n//2] for i in range(l//lstep)]

In [4]:
#returns configurations with domain wall in region of specified size, q = 2, n >= a
def configs_dw(n, a):
    result = {}
    configs = {format(i, '0'+str(n)+'b'): i for i in range(2**n)}
    for cii in configs:
        if ('0' in cii[:a]) and ('1' in cii[:a]):
            result[cii] = configs[cii]
    return result

In [13]:
#compute trace norm given output coefficients
def coeffs_trace_norm(c, q, n, select):
    #TODO
    if len(c) != 2**n:
        print('Dimensions must match!')
        return -1
    if len(select) == 0:
        return 0
    if len(select[0]) != n:
        print('Dimensions must match!')
        return -1
    #swap has q + q(q-1)/2 eigenvectors with +1 and q(q-1)/2 eigenvectors with -1
    return -1

#compute trace norm given output coefficients, q = 2
def coeffs_trace_norm_2(c, n, select):
    if len(c) != 2**n:
        print('Dimensions must match!')
        return -1
    if len(select) == 0:
        return 0
    #swap has 3 eigenvectors with +1 and one with -1
    evs = [format(np.base_repr(i, base=4), '>0' + str(n)) for i in range(4**n)]
    coeffs = {}
    coeffsum = 0
    for ci in select:
        if c[select[ci]] != 0:
            coeffs[ci] = c[select[ci]]
            coeffsum += c[select[ci]]
    result = 0
    #print(coeffsum)
    for ev in evs:
        if '3' in ev:
            cosum = 0
            for ci in coeffs:
                sign = 1
                for i in range(n):
                    if (ci[i] == '1') and (ev[i] == '3'):
                        sign *= -1
                cosum += sign * coeffs[ci]
            result += abs(cosum)
            #print(cosum)
        else:
            result += coeffsum
    return result

#compute trace norm given output coefficients, q = 2
def coeffs_trace_norm_2_i(c, n, select):
    if len(c) != 2**n:
        print('Dimensions must match!')
        return -1
    if len(select) == 0:
        return 0
    #swap has 3 eigenvectors with +1 and one with -1
    evs = [format(i, '0'+str(n)+'b') for i in range(2**n)]
    coeffs = {}
    for ci in select:
        if c[select[ci]] != 0:
            coeffs[ci] = c[select[ci]]
    result = 0
    for ev in evs:
        cosum = 0
        w = ev.count('1')
        for ci in coeffs:
            sign = 1
            for i in range(n):
                if ev[i] == '1' and ci[i] == '1':
                    sign *= -1
            cosum += sign * coeffs[ci]
        #print(w)
        result += (3**w) * abs(cosum)
        #print(cosum)
    return result

In [6]:
#compute coefficients for trapezoidal circuit
#a should be even, l >= 2, q >= 2
def coeffs_trap_product_state(q, a, l, lstep):
    #useful defs
    n = a + 2 * (l - 1)
    d = q**2
    w = q/(q**2 + 1)
    cn = 2**n
    cm = 2**(n//2)
    configs = {format(i, '0'+str(n)+'b'): i for i in range(cn)}
    lastconfig = {}
    for i in range(cm):
        rep = format(i, '0'+str(n//2)+'b')
        config = ''
        for j in range(n//2):
            config += rep[j] + rep[j]
        lastconfig[config] = configs[config]
    c = np.zeros(((l-1)*(n//2 + a//2 - 1)//2 + 1, cn))
    for cii in lastconfig:
        c[0][lastconfig[cii]] = (1/d/(d+1))**(n//2)
    offset = -1
    nextconfig = {}
    lsgates = [i for i in range(n//2 - 1)]
    j = 0
    for t in range(1, l):
        #print(t)
        offset *= -1
        for g in lsgates:
            j += 1
            #even layer: g, g-1 input
            #odd layer: g, g+1 input
            for config in lastconfig:
                if config[2*g + 1] == config[2*g + 1 + offset]:
                    nextconfig[config] = offset
                    c[j][configs[config]] += c[j - 1][configs[config]]
                elif offset > 0:
                    config1 = config[:2*g + 1] + '00' + config[2*g + 3:]
                    config2 = config[:2*g + 1] + '11' + config[2*g + 3:]
                    nextconfig[config1] = offset
                    nextconfig[config2] = offset
                    c[j][configs[config1]] += c[j - 1][configs[config]] * w
                    c[j][configs[config2]] += c[j - 1][configs[config]] * w
                else:
                    config1 = config[:2*g] + '00' + config[2*g + 2:]
                    config2 = config[:2*g] + '11' + config[2*g + 2:]
                    nextconfig[config1] = offset
                    nextconfig[config2] = offset
                    c[j][configs[config1]] += c[j - 1][configs[config]] * w
                    c[j][configs[config2]] += c[j - 1][configs[config]] * w
            #print(g)
            #print(lastconfig)
            #print(nextconfig)
            #print()
            lastconfig = nextconfig
            nextconfig = {}
        #print(lastconfig)
        #print(nextconfig)
        del lsgates[(offset - 1)//2]
        #print(lsgates)
    #print(lastconfig)
    #print(c[-1])
    #print(np.sum(c[-1]) - c[-1][0] - c[-1][-1])
    #print(j)
    return [c[lstep*i*(n - lstep*i - 1)//2] for i in range(l//lstep)]

In [ ]:
#compute coefficients for partial trace over output

In [183]:
q = 2
l = 10
lstep = 1
for n in [24]:
    c = coeffs_product_state(q, n, l, lstep)
    print(n)
    f = open('output/coeffs_product_state_' + str(n) + '_' + str(l) + '_' + str(lstep), 'w')
    for t in range(len(c)):
        f.writelines('%s\n' % ci for ci in c[t])
        f.write('\n')
    f.close()
#dw = configs_dw(n, a)

24


In [181]:
c = coeffs_product_state(2, 18, 6, 1)
dw = configs_dw(18, 6)
print(4**18 * np.sum([c[-1][dw[ci]] for ci in dw]))

c = coeffs_product_state(2, 20, 6, 1)
dw = configs_dw(20, 6)
print(4**20 * np.sum([c[-1][dw[ci]] for ci in dw]))

c = coeffs_product_state(2, 22, 6, 1)
dw = configs_dw(22, 6)
print(4**22 * np.sum([c[-1][dw[ci]] for ci in dw]))

1.4738071083256765
1.7755958156628664
2.1170770511079713


In [185]:
for n in range(10, 24, 2):
    for a in [4, 6, 8]:
        dw = configs_dw(n, a)
        with open('output/configs_dw_' + str(n) + '_' + str(a), 'w') as f:
            f.writelines('%s\n' % cii for cii in [dw[cii] for cii in dw])

In [190]:
c = coeffs_product_state(4, 18, 6, 1)
dw = configs_dw(18, 6)
print(16**18 * np.sum([c[-1][dw[ci]] for ci in dw]))

c = coeffs_product_state(4, 20, 6, 1)
dw = configs_dw(20, 6)
print(16**20 * np.sum([c[-1][dw[ci]] for ci in dw]))

c = coeffs_product_state(4, 22, 6, 1)
dw = configs_dw(22, 6)
print(16**22 * np.sum([c[-1][dw[ci]] for ci in dw]))

0.014408461967519395
0.016903618745898026
0.019402987597476287


In [234]:
c = coeffs_product_state(2, 8, 2, 1)[-1]
print(coeffs_trace_norm_2(c, 8, configs_dw(8, 4)))

c = coeffs_product_state(2, 8, 4, 1)[-1]
print(coeffs_trace_norm_2(c, 8, configs_dw(8, 4)))

c = coeffs_product_state(2, 8, 6, 1)[-1]
print(coeffs_trace_norm_2(c, 8, configs_dw(8, 4)))

c = coeffs_product_state(2, 10, 2, 1)[-1]
print(coeffs_trace_norm_2(c, 10, configs_dw(10, 4)))

c = coeffs_product_state(2, 10, 4, 1)[-1]
print(coeffs_trace_norm_2(c, 10, configs_dw(10, 4)))

c = coeffs_product_state(2, 10, 6, 1)[-1]
print(coeffs_trace_norm_2(c, 10, configs_dw(10, 4)))

0.5964160000000114
0.17900205768695882
0.0534174322070119
0.6684159999975224
0.2317571194886744
0.07801364020407685


In [238]:
c = coeffs_product_state(2, 8, 2, 1)[-1]
dw = configs_dw(8, 4)
print(np.sum([c[dw[ci]] for ci in dw]) * 4**8)

c = coeffs_product_state(2, 8, 4, 1)[-1]
dw = configs_dw(8, 4)
print(np.sum([c[dw[ci]] for ci in dw]) * 4**8)

c = coeffs_product_state(2, 8, 6, 1)[-1]
dw = configs_dw(8, 4)
print(np.sum([c[dw[ci]] for ci in dw]) * 4**8)

c = coeffs_product_state(2, 10, 2, 1)[-1]
dw = configs_dw(10, 4)
print(np.sum([c[dw[ci]] for ci in dw]) * 4**10)

c = coeffs_product_state(2, 10, 4, 1)[-1]
dw = configs_dw(10, 4)
print(np.sum([c[dw[ci]] for ci in dw]) * 4**10)

c = coeffs_product_state(2, 10, 6, 1)[-1]
dw = configs_dw(10, 4)
print(np.sum([c[dw[ci]] for ci in dw]) * 4**10)

2.479882240000001
0.7331556418912262
0.21875936041865396
3.628072960000002
1.1251311076966406
0.3750882323283421


In [239]:
c = coeffs_product_state(2, 12, 2, 1)[-1]
print(coeffs_trace_norm_2(c, 12, configs_dw(12, 4)))

c = coeffs_product_state(2, 14, 2, 1)[-1]
print(coeffs_trace_norm_2(c, 14, configs_dw(14, 4)))

0.6868134399893335


KeyboardInterrupt: 

In [72]:
c = coeffs_trap_product_state(2, 4, 2, 1)
dw = configs_dw(6, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 4**6)
#print(coeffs_trace_norm_2(c[-1], 6, dw))

c = coeffs_trap_product_state(2, 4, 4, 1)
dw = configs_dw(10, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 4**10)
#print(coeffs_trace_norm_2(c[-1], 10, dw))

c = coeffs_trap_product_state(2, 4, 6, 1)
dw = configs_dw(14, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 4**14)

c = coeffs_trap_product_state(2, 4, 8, 1)
dw = configs_dw(18, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 4**18)

print()

c = coeffs_trap_product_state(3, 4, 2, 1)
dw = configs_dw(6, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 9**6)

c = coeffs_trap_product_state(3, 4, 4, 1)
dw = configs_dw(10, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 9**10)

c = coeffs_trap_product_state(3, 4, 6, 1)
dw = configs_dw(14, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 9**14)

c = coeffs_trap_product_state(3, 4, 8, 1)
dw = configs_dw(18, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 9**18)

print()

c = coeffs_trap_product_state(4, 4, 2, 1)
dw = configs_dw(6, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 16**6)

c = coeffs_trap_product_state(4, 4, 4, 1)
dw = configs_dw(10, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 16**10)

c = coeffs_trap_product_state(4, 4, 6, 1)
dw = configs_dw(14, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 16**14)

c = coeffs_trap_product_state(4, 4, 8, 1)
dw = configs_dw(18, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 16**18)

c = coeffs_trap_product_state(4, 4, 10, 1)
dw = configs_dw(22, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 16**22)

print()

c = coeffs_trap_product_state(8, 4, 2, 1)
dw = configs_dw(6, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 64**6)

c = coeffs_trap_product_state(8, 4, 4, 1)
dw = configs_dw(10, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 64**10)

c = coeffs_trap_product_state(8, 4, 6, 1)
dw = configs_dw(14, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 64**14)

c = coeffs_trap_product_state(8, 4, 8, 1)
dw = configs_dw(18, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 64**18)

c = coeffs_trap_product_state(8, 4, 10, 1)
dw = configs_dw(22, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 64**22)

1.7203200000000005
2.205236865269761
2.9469172461730153
3.5612975931941113

1.7058599999999995
1.544320793363759
1.6784264043721584
1.722517723867893

1.4539379669924506
1.0156160351072585
1.0298293915926116
1.030273889736587
1.0300990286498986

0.7916573280939851
0.36521238941521456
0.3630372535678201
0.3628582542441066
0.3628464920064653


In [55]:
c = coeffs_product_state(2, 14, 2, 1)[-1]
dw = configs_dw(14, 4)
print(np.sum([c[dw[ci]] for ci in dw]) * 4**14)

c = coeffs_product_state(2, 14, 3, 1)[-1]
dw = configs_dw(14, 4)
print(np.sum([c[dw[ci]] for ci in dw]) * 4**14)

c = coeffs_product_state(2, 14, 4, 1)[-1]
dw = configs_dw(14, 4)
print(np.sum([c[dw[ci]] for ci in dw]) * 4**14)

c = coeffs_product_state(2, 14, 5, 1)[-1]
dw = configs_dw(14, 4)
print(np.sum([c[dw[ci]] for ci in dw]) * 4**14)

c = coeffs_product_state(2, 14, 6, 1)[-1]
dw = configs_dw(14, 4)
print(np.sum([c[dw[ci]] for ci in dw]) * 4**14)

7.537774978662405
2.0342520373053503
2.003475038446715
0.6210692236753136
0.6934553422794508


In [71]:
q = 5
d = q**2
c = coeffs_trap_product_state(q, 4, 2, 1)
dw = configs_dw(6, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * d**6)

c = coeffs_trap_product_state(q, 4, 4, 1)
dw = configs_dw(10, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * d**10)

c = coeffs_trap_product_state(q, 4, 6, 1)
dw = configs_dw(14, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * d**14)

c = coeffs_trap_product_state(q, 4, 8, 1)
dw = configs_dw(18, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * d**18)

c = coeffs_trap_product_state(q, 4, 10, 1)
dw = configs_dw(22, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * d**22)

c = coeffs_trap_product_state(q, 4, 12, 1)
dw = configs_dw(26, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * d**26)

1.2230275348579154
0.7220146323083957
0.7192715036162945
0.718018888048598


KeyboardInterrupt: 

In [76]:
q = 4
c = coeffs_trap_product_state(q, 4, 2, 1)
dw = configs_dw(6, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 16**6)
print(trap_bound(q, 4, 2))
print()

c = coeffs_trap_product_state(q, 4, 4, 1)
dw = configs_dw(10, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 16**10)
print(trap_bound(q, 4, 4))
print()

c = coeffs_trap_product_state(q, 4, 6, 1)
dw = configs_dw(14, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 16**14)
print(trap_bound(q, 4, 6))
print()

c = coeffs_trap_product_state(q, 4, 8, 1)
dw = configs_dw(18, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 16**18)
print(trap_bound(q, 4, 8))
print()

c = coeffs_trap_product_state(q, 4, 10, 1)
dw = configs_dw(22, 4)
print(np.sum([c[-1][dw[ci]] for ci in dw]) * 16**22)
print(trap_bound(q, 4, 10))
print()

1.4539379669924506
194.78712557502328

1.0156160351072585
21.711725800546134

1.0298293915926116
14.316385110973101

1.030273889736587
10.849940346129092

1.0300990286498986
8.44966623726645



In [79]:
for l in range(12, 36, 6):
    print(trap_bound(q, 4, l))

6.619154490337659
3.196336779517964
1.5441982870706867
0.7460291166225692


In [ ]:
c = coeffs_product_state(2, 10, 2, 1)[-1]
print(coeffs_trace_norm_2(c, 10, configs_dw(10, 4)))
print(coeffs_trace_norm_2_i(c, 10, configs_dw(10, 4)))